In [1]:
# install syft if we are on google colab
%load_ext autoreload
%autoreload 2
import syft as sy
print(f"You're running syft version: {sy.__version__}")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from tqdm import tqdm

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You're running syft version: 0.7.0-beta.20


In [2]:
domain_client = sy.login(email='info@openmined.org', password='changethis',port = 8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [3]:
data = domain_client.datasets[-1]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

# Preprocess the images

X_train = X_train.T
X_train = X_train / 255.0

X_dev = X_dev.T
X_dev = X_dev / 255.0

/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:75: RuntimeWarning: invalid value encountered in true_divide
  res = (getattr(np.empty(x_shape), op_str)(np.empty(y_shape))).shape


In [4]:

def init_params():
    W1 = np.random.rand(6, 4096) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return Z*(Z>0)

def softmax(Z):
    exp_cache = Z.exp()
    A = exp_cache * (exp_cache.sum().reciprocal())
    return A
    
def forward_prop(W1, b1, W2, b2, X):    
    Z1 = X.__rmatmul__(W1) + b1
    A1 = ReLU(Z1)
    Z2 = A1.__rmatmul__(W2) + b2
    A2 = softmax(Z2) 
    return Z1, A1, Z2, A2


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2@(A1.T)
    db2 = 1 / m * dZ2.sum()
    dZ1 = W2.T@(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1@(X.T)
    db1 = 1 / m * dZ1.sum()
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2


def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        # dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        # W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # if i % 40 == 0:
        #     print("Predicition will be availabe in the next iteration.....Thank you for your support.")
        #     # predictions = get_predictions(A2)
        #     # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))
    return Z1, A1, Z2, A2
    return W1, b1, W2, b2

In [ ]:
Z1, A1, Z2, A2 = gradient_descent(X_train, Y_train, 0.10, 1)

In [20]:
Z = X_train

In [ ]:
v1 = Z.get_copy()

In [21]:
exp_cache = Z.exp()
# A = exp_cache * (exp_cache.sum().reciprocal())

In [22]:
v1 = exp_cache.sum().reciprocal()

In [18]:
v2 = exp_cache * v1  

In [ ]:
t1 = exp_cache.get_copy()

In [ ]:
t2 = v1.get_copy()

In [ ]:
type(t1.child)
type(t2.child)

In [ ]:
W1, b1, W2, b2 = init_params()

In [ ]:
v1 = X_train.__rmatmul__(W1)

In [ ]:
np.array([1]).send(domain_client)

In [ ]:
import numpy as np

In [ ]:
a = np.array([1,2,3,4,9,10])

In [ ]:
one_hot(a).shape